<a href="https://colab.research.google.com/github/YT1202/DSP-Introduction/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!nvidia-smi

Fri Jan  8 14:13:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
import time
import scipy
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten
from keras.layers import Dropout
from scipy import signal

In [ ]:
# 計時開始
tStart = time.time()

In [ ]:
# Step_1 載入資料集  
sample, label = np.load("traindata.npy"), np.load("trainlabel.npy")
val_sample, val_label = np.load("validationdata.npy"), np.load("validationlabel.npy")
test_sample = np.load("testdata.npy")
print("Train size: ", sample.shape)
print("Train target: ", label.shape, "\n-------------------------------------")
print("Val size: ", val_sample.shape)
print("Val target: ", val_label.shape, "\n-------------------------------------")
print("Test size: ", test_sample.shape, "\n=======================================")

In [ ]:
# Step_2  時頻分析
f, t, spec = signal.stft(sample, fs = 16000)
f, t, val_spec = signal.stft(val_sample, fs = 16000)
f, t, test_spec = signal.stft(test_sample, fs = 16000)
print("Spec size: ", spec.shape, "\n-------------------------------------")
print("Val_Spec size: ", val_spec.shape, "\n-------------------------------------")
print("Test_Spec size: ", test_spec.shape, "\n-------------------------------------")
x_train = np.abs(spec.reshape(-1, 2, 129, 158))
x_val = np.abs(val_spec.reshape(-1, 2, 129, 158))
x_test = np.abs(test_spec.reshape(-1, 2, 129, 158))
print("x_train size: ", x_train.shape, "\n-------------------------------------")
print("x_val size: ",x_val.shape, "\n-------------------------------------")
print("x_test size: ", x_test.shape, "\n=======================================")

plt.subplot(2,1,1)
plt.imshow(x_train[0][0], aspect='auto')
plt.subplot(2,1,2)
plt.imshow(x_train[0][1], aspect='auto')

In [ ]:
# Step_3  利用OneHot對label進行編碼
y_train = label
y_val = val_label

y_train_onehot = np_utils.to_categorical(y_train)
y_val_onehot = np_utils.to_categorical(y_val)

In [ ]:
# Step_4  建立訓練模型
model = Sequential()

# Conv layer 1 output shape (6, 129, 158)
model.add(Convolution2D(
     input_shape=(129, 158, 2),
     filters=6,
     kernel_size=5,
     strides=1,
     padding='same',
     activation='relu'))

# Pooling layer 1 (max pooling) output shape (32, 65, 79)
model.add(MaxPooling2D(
     pool_size=2,
     strides=2,
     padding='same'))

# Conv layer 2 output shape (16, 65, 79)
model.add(Convolution2D(filter=16, 
            kernel_size=5, 
            strides=1, 
            padding='same', 
            activation='relu'))

# Pooling layer 2 (max pooling) output shape (16, 33, 40)
model.add(MaxPooling2D(
     pool_size=2,
     strides=2,
     padding='same'))

# Conv layer 3 output shape (16, 33, 40)
model.add(Convolution2D(filter=16, 
            kernel_size=5, 
            strides=1, 
            padding='same', 
            activation='relu'))

# Pooling layer 3 (max pooling) output shape (16, 17, 20)
model.add(MaxPooling2D(
     pool_size=2,
     strides=2,
     padding='same'))

# Conv layer 4 output shape (36, 17, 20)
model.add(Convolution2D(filter=36, 
            kernel_size=5, 
            strides=1, 
            padding='same', 
            activation='relu'))

# Pooling layer 4 (max pooling) output shape (36, 9, 10)
model.add(MaxPooling2D(
     pool_size=2,
     strides=2,
     padding='same'))

# Conv layer 5 output shape (48, 9, 10)
model.add(Convolution2D(filter=48, 
            kernel_size=5, 
            strides=1, 
            padding='same', 
            activation='relu'))

# Pooling layer 5 (max pooling) output shape (48, 5, 5)
model.add(MaxPooling2D(
     pool_size=2,
     strides=2,
     padding='same'))

model.add(Dropout(0.25))

# Fc layer 1 input shape (48 * 5 * 5) = (1200), output shape (1024)
model.add(Flatten())
model.add(Dense(units=1024, 
        activation='relu'))
model.add(Dropout(0.5))
# Fc layer 2 input shape (1024), output shape (128)
model.add(Dense(units=128,  
        activation='relu'))
model.add(Dropout(0.5))
# Output
model.add(Dense(units=4,  
        activation='softmax'))
print(model.summary()) 

In [ ]:
# Step_4  參數設定
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

train_history=model.fit(x=x_train, 
            y=y_train_onehot,
            validation_data=(x_val, y_val_onehot), 
            epochs=5, 
            batch_size=64,
            verbose=2)

In [ ]:
# Step_5  顯示訓練結果: Accurancy & Loss
def show_train_history(train_history,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

show_train_history(train_history,'accuracy','val_accuracy')
show_train_history(train_history,'loss','val_loss')

In [ ]:
# Step_6 儲存模型
model.save('model.h5') 

In [ ]:
# Step_7  用測試集檢驗模型
scores = model.evaluate(x_val, y_val_onehot)
print('accurancy= ', scores[1])

In [ ]:
# Step_8  利用訓練好的模型驗證
prediction = model.predict_classes(x_test)
y_test = prediction.reshape(-1,1)

In [ ]:
# Step_9  產生繳交csv格式 
id_test = np.array(range(0, 960)).reshape(-1,1)
submission = np.hstack([id_test, y_test])
submission = pd.DataFrame(submission, columns=["id", "category"])

submission.to_csv('submission.csv', sep=',', index=False)

In [ ]:
# 計時結束
tEnd = time.time()
print('==========================================\ntime : ', tEnd - tStart, '(s)')